<a href="https://colab.research.google.com/github/RemyaVKarthikeyan/AA-Stagecoach-Project/blob/main/Final_Timeseries_forecasting_6_Aug_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
from difflib import SequenceMatcher

# Function to normalize stop names
def normalize_stop_name(name):
    return ' '.join(name.lower().split())

from google.colab import drive
drive.mount('/content/drive')

# Modify this path accordingly
file_path = '/content/drive/MyDrive/QSI points.xlsx'

# Read the Excel file into a DataFrame
df = pd.read_excel(file_path)

# Get the lineID from the user
lineID = input("Please enter the lineID: ")

# Check if the necessary column is present
if 'Route_Dir_QSI_No' not in df.columns:
    print("The 'Route_Dir_QSI_No' column is not present in the provided file.")
else:
    # Convert the lineID to uppercase to ensure case-insensitivity
    lineID = lineID.upper()

    # Convert the 'Route_Dir_QSI_No' column to uppercase for comparison
    df['Route_Dir_QSI_No'] = df['Route_Dir_QSI_No'].str.upper()

    # Normalize the stop names in the DataFrame
    df['STOP_NAME'] = df['STOP_NAME'].apply(normalize_stop_name)

    # Create regular expressions for filtering
    pattern_A = f"^{lineID}_A\\d+$"  # Regular expression for lineID_A**
    pattern_B = f"^{lineID}_B\\d+$"  # Regular expression for lineID_B**

    # Filter rows where the 'Route_Dir_QSI_No' column matches the pattern
    filtered_df_A = df[df['Route_Dir_QSI_No'].str.match(pattern_A, na=False)][['Route_Dir_QSI_No', 'STOP_NAME']]
    filtered_df_B = df[df['Route_Dir_QSI_No'].str.match(pattern_B, na=False)][['Route_Dir_QSI_No', 'STOP_NAME']]

    # Function to fetch and process route sequence data from TfL API
    def fetch_and_process_route_data(route_type, pattern, filtered_df):
        api_url = f"https://api.tfl.gov.uk/Line/{lineID}/Route/Sequence/{route_type}"
        response = requests.get(api_url)

        results_list = []

        if response.status_code == 200:
            route_data = response.json()

            # Iterate through each stop in the route data
            for stop in route_data['stopPointSequences'][0]['stopPoint']:
                stop_name_api = normalize_stop_name(stop['name'])
                stop_id = stop['id']

                # Check if the stop_name_api exists in the filtered DataFrame for the correct direction
                matched_row = filtered_df[(filtered_df['STOP_NAME'] == stop_name_api) &
                                          (filtered_df['Route_Dir_QSI_No'].str.match(pattern))]

                if not matched_row.empty:
                    route_dir_qsi_no = matched_row.iloc[0]['Route_Dir_QSI_No']
                    results_list.append({
                        'Route_Dir_QSI_No': route_dir_qsi_no,
                        'STOP_Name': stop['name'],
                        'ID': stop_id
                    })
                else:
                    # If exact match not found, try partial matching based on words before and after '/'
                    api_stop_name_parts = stop_name_api.split('/')
                    for index, row in filtered_df.iterrows():
                        df_stop_name_parts = row['STOP_NAME'].split('/')
                        for api_part in api_stop_name_parts:
                            for df_part in df_stop_name_parts:
                                if SequenceMatcher(None, df_part.strip(), api_part.strip()).ratio() > 0.8:
                                    matched_row = pd.DataFrame([row])
                                    break
                            if not matched_row.empty:
                                break
                        if not matched_row.empty:
                            break

                    if not matched_row.empty:
                        route_dir_qsi_no = matched_row.iloc[0]['Route_Dir_QSI_No']
                        results_list.append({
                            'Route_Dir_QSI_No': route_dir_qsi_no,
                            'STOP_Name': stop['name'],
                            'ID': stop_id
                        })
        else:
            print(f"Failed to fetch route sequence data from TfL API for {route_type} route. Status code: {response.status_code}")

        return results_list

    # Fetch and process outbound route data for _A**
    matched_results_A = fetch_and_process_route_data('outbound', pattern_A, filtered_df_A)

    # Fetch and process inbound route data for _B**
    matched_results_B = fetch_and_process_route_data('inbound', pattern_B, filtered_df_B)

    # Create DataFrames from the matched results for each direction
    matched_results_df_A = pd.DataFrame(matched_results_A)
    matched_results_df_B = pd.DataFrame(matched_results_B)

    # Function to remove partial matches if exact matches are found
    def remove_partial_matches(exact_df, matched_df):
        for index, row in exact_df.iterrows():
            exact_stop_name = row['STOP_NAME']
            route_dir_qsi_no = row['Route_Dir_QSI_No']
            # Find exact matches in matched_df
            exact_matches = matched_df[(matched_df['Route_Dir_QSI_No'] == route_dir_qsi_no) &
                                       (matched_df['STOP_Name'].apply(normalize_stop_name) == exact_stop_name)]
            if not exact_matches.empty:
                # Remove partial matches
                matched_df = matched_df[~((matched_df['Route_Dir_QSI_No'] == route_dir_qsi_no) &
                                          (matched_df['STOP_Name'].apply(normalize_stop_name) != exact_stop_name))]
        return matched_df

    # Remove partial matches for direction A
    matched_results_df_A = remove_partial_matches(filtered_df_A, matched_results_df_A)

    # Remove partial matches for direction B
    matched_results_df_B = remove_partial_matches(filtered_df_B, matched_results_df_B)

    # Remove duplicate stop names with the same Route_Dir_QSI_No and different IDs
    matched_results_df_A = matched_results_df_A.drop_duplicates(subset=['Route_Dir_QSI_No', 'STOP_Name'], keep='first')
    matched_results_df_B = matched_results_df_B.drop_duplicates(subset=['Route_Dir_QSI_No', 'STOP_Name'], keep='first')

    # Print the matched results for direction A
    print(f"\n\n\033[1m\033[4mQSI stop points for direction {lineID}_A\033[0m\n")
    matched_results_df_A = matched_results_df_A[matched_results_df_A['Route_Dir_QSI_No'].str.match(pattern_A)]
    print(matched_results_df_A[['Route_Dir_QSI_No', 'STOP_Name', 'ID']])

    # Print the matched results for direction B
    print(f"\n\n\033[1m\033[4mQSI stop points for direction {lineID}_B\033[0m\n")
    matched_results_df_B = matched_results_df_B[matched_results_df_B['Route_Dir_QSI_No'].str.match(pattern_B)]
    print(matched_results_df_B[['Route_Dir_QSI_No', 'STOP_Name', 'ID']])

    # Concatenate the matched results DataFrames for directions A and B
    combined_df = pd.concat([matched_results_df_A, matched_results_df_B], ignore_index=True)

    # Display the combined DataFrame
    print("\n\n\033[1m\033[4mCombined QSI stop points for directions A and B\033[0m\n")
    print(combined_df[['Route_Dir_QSI_No', 'STOP_Name', 'ID']])


Mounted at /content/drive
Please enter the lineID: d7


QSI stop points for direction D7_A

  Route_Dir_QSI_No                    STOP_Name          ID
0            D7_A1   Poplar / All Saints Church  490011107G
1            D7_A2               Stewart Street  490013513S
2            D7_A3       Island Gardens Station  490002048Z
3            D7_A4  Arnhem Wharf Primary School  490006092N
5            D7_A5         East India Dock Road  490004584N


QSI stop points for direction D7_B

  Route_Dir_QSI_No                    STOP_Name          ID
0            D7_B1  Mile End Station / Bow Road  490015151H
2            D7_B2         East India Dock Road  490004584S
3            D7_B3         Canary Wharf Station  490000038F
5            D7_B4  Arnhem Wharf Primary School  490006092S
6            D7_B5       Island Gardens Station  490002048X
7            D7_B6               Stewart Street  490013513N


Combined QSI stop points for directions A and B

   Route_Dir_QSI_No                    

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import pytz
import time
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# Function to fetch arrival predictions
def fetch_arrival_predictions(line_id, stop_point_id, direction):
    try:
        base_url = f"https://api.tfl.gov.uk/Line/{line_id}/Arrivals/{stop_point_id}"
        params = {'direction': direction}
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()
        if len(data) == 0:
            return pd.DataFrame(), None  # No data available
        station_name = data[0]['stationName']
        predictions = []
        for item in data:
            arrival_time = datetime.strptime(item['expectedArrival'], '%Y-%m-%dT%H:%M:%SZ')
            arrival_time_bst = arrival_time + timedelta(hours=1)
            predictions.append({
                'Line': item['lineName'],
                'Vehicle ID': item['vehicleId'],
                'Stop Point': stop_point_id,
                'Direction': direction,
                'Expected Arrival (BST)': arrival_time_bst,
                'Expected Arrival (HM)': arrival_time_bst.strftime('%H:%M')
            })
        df = pd.DataFrame(predictions)
        df = df.sort_values(by='Expected Arrival (BST)', ascending=True)
        df['Expected Arrival (BST)'] = pd.to_datetime(df['Expected Arrival (BST)'])  # Convert to datetime
        df['Expected Arrival (HM)'] = pd.to_datetime(df['Expected Arrival (HM)'], format='%H:%M')
        df['Gap'] = df['Expected Arrival (HM)'].diff().fillna(pd.Timedelta(seconds=0)).dt.total_seconds() / 60
        df['2_Gap'] = (df['Gap'] * 2).round(2)
        df['Gap_Sq'] = (df['Gap'] * df['Gap']).round(2)
        return df, station_name
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None, None

def main():
    # Assuming combined_df is generated from the previous cell
    # combined_df should have columns like 'Route_Dir_QSI_No', 'STOP_Name', 'ID'
    import pandas as pd
    import requests
    from datetime import datetime
    import pytz

    # Function to normalize stop names
    def normalize_stop_name(name):
        return ' '.join(name.lower().split())

    # Function to fetch data from the API
    def fetch_data(url):
        response = requests.get(url)
        return response.json()

    # Function to extract schedule names
    def extract_schedule_names(data, schedule_names_dict={}):
        if isinstance(data, dict):
            if data.get('$type') == "Tfl.Api.Presentation.Entities.Schedule, Tfl.Api.Presentation.Entities" and 'knownJourneys' in data:
                if 'name' in data:
                    schedule_names_dict[data['name']] = data['knownJourneys']
            for key, value in data.items():
                extract_schedule_names(value, schedule_names_dict)
        elif isinstance(data, list):
            for item in data:
                extract_schedule_names(item, schedule_names_dict)
        return schedule_names_dict

    # Function to categorize journeys into hourly slots
    def categorize_into_slots(timetable):
        slots = [[] for _ in range(24)]
        for journey in timetable:
            hour = int(journey['hour'])  # Convert hour to integer
            if 0 <= hour < 24:  # Ensure hour is within the valid range
                slots[hour].append(journey)
        return slots

    # Function to fetch the current day of the week
    def get_day_of_week():
        bst = pytz.timezone('Europe/London')
        now = datetime.now(bst)
        return now.strftime('%A')  # %A gives full weekday name (e.g., 'Monday')

    # Function to calculate Scheduled Wait Time (SWT) for all hours
    def calculate_swt_for_all_hours(slots):
        swt_per_hour = []
        for hour in range(24):
            total_buses = len(slots[hour])
            scheduled_wait_time = 60 / (total_buses * 2) if total_buses > 0 else None  # Use None to indicate no buses
            swt_per_hour.append((scheduled_wait_time, total_buses))
        return swt_per_hour

    # Function to select the preferred schedule name based on the current day of the week
    def select_preferred_schedule(schedule_names_dict, day_of_week):
        if day_of_week.lower() in ['monday', 'tuesday', 'wednesday', 'thursday']:
            preferred_schedule_names = ['Mon-Fri Schooldays', 'Monday to Thursday', 'Monday to Friday']
        elif day_of_week.lower() == 'friday':
            preferred_schedule_names = ['Mon-Fri Schooldays', 'Monday to Friday', 'Friday']
        elif day_of_week.lower() == 'saturday':
            preferred_schedule_names = ['Saturday']
        elif day_of_week.lower() == 'sunday':
            preferred_schedule_names = ['Sunday']
        else:
            preferred_schedule_names = [day_of_week]

        for preferred_name in preferred_schedule_names:
            if preferred_name in schedule_names_dict:
                return preferred_name
        return None

    # Main logic to fetch and calculate SWT
    def main(combined_df, lineID):
        bst = pytz.timezone('Europe/London')
        swt_data = {
            'Route_Dir_QSI_No': [],
            'ID': [],
        }
        # Initialize keys for all 24 hours in the dictionary
        for hour in range(24):
            swt_data[f'SWT_{hour}'] = []
            swt_data[f'Sch_{hour}'] = []

        # Update current time and hour
        current_time = datetime.now(bst)
        day_of_week = get_day_of_week()

        # Store selected schedule name to ensure it's printed only once
        selected_schedule_name = None
        printed_schedule_name = False

        for index, row in combined_df.iterrows():
            stop_point_id = row['ID']
            route_dir_qsi_no = row['Route_Dir_QSI_No']

            if f"{lineID}_A" in route_dir_qsi_no:
                direction = 'outbound'
            elif f"{lineID}_B" in route_dir_qsi_no:
                direction = 'inbound'
            else:
                continue

            url = f'https://api.tfl.gov.uk/Line/{lineID}/Timetable/{stop_point_id}?direction={direction}'
            data = fetch_data(url)

            schedule_names_dict = extract_schedule_names(data)

            if not selected_schedule_name:
                selected_schedule_name = select_preferred_schedule(schedule_names_dict, day_of_week)

            if selected_schedule_name and not printed_schedule_name:
                print(f"\n\033[1m\033[4mToday is {day_of_week}. The selected Schedule name is {selected_schedule_name}.\033[0m")
                printed_schedule_name = True

            if selected_schedule_name:
                timetable = schedule_names_dict[selected_schedule_name]
                slots = categorize_into_slots(timetable)

                # Calculate SWT for all hours
                swt_per_hour = calculate_swt_for_all_hours(slots)

                # Store SWT data for all hours
                swt_data['Route_Dir_QSI_No'].append(route_dir_qsi_no)
                swt_data['ID'].append(stop_point_id)
                for hour in range(24):
                    swt, total_buses = swt_per_hour[hour]
                    swt_data[f'SWT_{hour}'].append(swt)
                    swt_data[f'Sch_{hour}'].append(total_buses)

        # Create DataFrame for SWT data
        swt_df = pd.DataFrame(swt_data)

        # Calculate Route SWT for each hour for directions A and B
        route_swt_data = {
            'Hour': [],
            'Route SWT A': [],
            'Route SWT B': []
        }

        for hour in range(24):
            # Calculate Route SWT for direction A
            swt_a = swt_df[swt_df['Route_Dir_QSI_No'].str.contains(f'{lineID}_A')]
            valid_swt_a = swt_a[pd.notna(swt_a[f'SWT_{hour}'])]  # Filter out NaN values
            weighted_sum_a = sum(valid_swt_a[f'SWT_{hour}'] * valid_swt_a[f'Sch_{hour}']) if not valid_swt_a.empty else 0
            total_buses_a = sum(valid_swt_a[f'Sch_{hour}']) if not valid_swt_a.empty else 0
            route_swt_a = round(weighted_sum_a / total_buses_a, 2) if total_buses_a > 0 else None


            # Calculate Route SWT for direction B
            swt_b = swt_df[swt_df['Route_Dir_QSI_No'].str.contains(f'{lineID}_B')]
            valid_swt_b = swt_b[pd.notna(swt_b[f'SWT_{hour}'])]  # Filter out NaN values
            weighted_sum_b = sum(valid_swt_b[f'SWT_{hour}'] * valid_swt_b[f'Sch_{hour}']) if not valid_swt_b.empty else 0
            total_buses_b = sum(valid_swt_b[f'Sch_{hour}']) if not valid_swt_b.empty else 0
            route_swt_b = round(weighted_sum_b / total_buses_b, 2) if total_buses_b > 0 else None

            route_swt_data['Hour'].append(hour)
            route_swt_data['Route SWT A'].append(route_swt_a)
            route_swt_data['Route SWT B'].append(route_swt_b)

        route_swt_df = pd.DataFrame(route_swt_data)
        return route_swt_df
        # Print the Route SWT DataFrame
        print("\n\nRoute SWT DataFrame:")
        print(route_swt_df.to_string())

    # Assuming combined_df and lineID are defined and available
    # combined_df = ... (from previous code)
    # lineID = ... (from user input)
    route_swt_df = main(combined_df, lineID)
    # Example dictionary to hold cumulative dataframes for each stop point
    cumulative_dataframes = {}

    # Dictionary to hold the number of buses observed per stop point
    buses_observed = {}

    # DataFrame to store Route AWT data
    route_awt_df = pd.DataFrame(columns=['Hour', 'Route AWT A', 'Route AWT B'])

    # Loop through unique stop points in combined_df
    for index, row in combined_df.iterrows():
        stop_point_id = row['ID']
        direction = 'outbound' if row['Route_Dir_QSI_No'].startswith(lineID + '_A') else 'inbound'

        cumulative_df = pd.DataFrame(columns=[
            'Line', 'Vehicle ID', 'Stop Point', 'Direction',
            'Expected Arrival (BST)', 'Expected Arrival (HM)',
            'Gap', '2_Gap', 'Gap_Sq'
        ])

        cumulative_dataframes[stop_point_id] = cumulative_df  # Initialize cumulative dataframe
        buses_observed[stop_point_id] = (0, 0, 0, 0)  # Initialize with zero values

    while True:
        for stop_point_id, cumulative_df in cumulative_dataframes.items():
            direction = 'outbound' if combined_df[combined_df['ID'] == stop_point_id]['Route_Dir_QSI_No'].iloc[0].startswith(lineID + '_A') else 'inbound'

            arrival_predictions_df, station_name = fetch_arrival_predictions(lineID, stop_point_id, direction)

            if arrival_predictions_df is not None and not arrival_predictions_df.empty:
                current_hour = datetime.now(pytz.timezone('Europe/London')).hour

                for _, row in arrival_predictions_df.iterrows():
                    vehicle_id = row['Vehicle ID']

                    mask = cumulative_df['Vehicle ID'] == vehicle_id

                    if cumulative_df[mask].empty:
                        # If vehicle ID is not present in cumulative DataFrame, append the row
                        cumulative_df = pd.concat([cumulative_df, row.to_frame().T], ignore_index=True)
                    else:
                        # If vehicle ID is present, overwrite the row
                        cumulative_df.loc[mask, ['Line', 'Stop Point', 'Direction', 'Expected Arrival (BST)', 'Expected Arrival (HM)', 'Gap', '2_Gap', 'Gap_Sq']] = row[['Line', 'Stop Point', 'Direction', 'Expected Arrival (BST)', 'Expected Arrival (HM)', 'Gap', '2_Gap', 'Gap_Sq']].values

                # Convert to datetime
                cumulative_df['Expected Arrival (BST)'] = pd.to_datetime(cumulative_df['Expected Arrival (BST)'])
                cumulative_df['Expected Arrival (HM)'] = pd.to_datetime(cumulative_df['Expected Arrival (HM)'], format='%H:%M')

                # Sort the DataFrame
                cumulative_df = cumulative_df.sort_values(by='Expected Arrival (BST)', ascending=True).reset_index(drop=True)

                # Calculate gaps
                cumulative_df['Gap'] = (cumulative_df['Expected Arrival (BST)'].diff().dt.total_seconds() / 60).round(2)
                cumulative_df.loc[0, 'Gap'] = 0  # First row gap should be zero
                cumulative_df['2_Gap'] = (cumulative_df['Gap'] * 2).round(2)
                cumulative_df['Gap_Sq'] = (cumulative_df['Gap'] * cumulative_df['Gap']).round(2)

                # Update number of buses observed in the current hour
                num_buses_observed = len(cumulative_df[cumulative_df['Expected Arrival (BST)'].dt.hour == current_hour]['Vehicle ID'].unique())
                total_Gap_Sq = cumulative_df[cumulative_df['Expected Arrival (BST)'].dt.hour == current_hour]['Gap_Sq'].sum()
                total_2_Gap = cumulative_df[cumulative_df['Expected Arrival (BST)'].dt.hour == current_hour]['2_Gap'].sum()
                AWT = round(total_Gap_Sq / total_2_Gap, 2) if total_2_Gap > 0 else 0
                buses_observed[stop_point_id] = (num_buses_observed, total_Gap_Sq, total_2_Gap, AWT)

                print(f"\nArrival Predictions for stop point {stop_point_id} ({station_name}):")
                print(arrival_predictions_df.to_string(index=False))
                print("\nCumulative DataFrame:")
                print(cumulative_df.to_string(index=False))
                print(f"\nNumber of buses observed in the current hour: {num_buses_observed}")
            else:
                print("No arrival predictions available.")



            print("Refreshing data in 30 seconds...\n")
            time.sleep(30)

            # Update cumulative dataframe in dictionary
            cumulative_dataframes[stop_point_id] = cumulative_df

        print("\nRoute SWT DataFrame:")
        print(route_swt_df)

        # Create DataFrame to show number of buses observed for each stop point
        buses_observed_df = pd.DataFrame(list(buses_observed.items()), columns=['Stop Point', 'Metrics'])

        # Split 'Metrics' into separate columns
        buses_observed_df[['Num of Buses Observed', 'Total Gap Sq', 'Total 2 Gap', 'AWT']] = pd.DataFrame(
            buses_observed_df['Metrics'].tolist(), index=buses_observed_df.index
        )

        # Calculate WAWT as the product of AWT and Num of Buses Observed
        buses_observed_df['WAWT'] = buses_observed_df['AWT'] * buses_observed_df['Num of Buses Observed']

        # Drop the 'Metrics' column
        buses_observed_df.drop(columns=['Metrics'], inplace=True)

        print("\nNumber of Buses Observed DataFrame:")
        print(buses_observed_df)

        # Calculate Route AWT A and Route AWT B
        pattern_A = f"^{lineID}_A\\d+$"
        pattern_B = f"^{lineID}_B\\d+$"

        buses_observed_df_A = buses_observed_df[buses_observed_df['Stop Point'].isin(combined_df[combined_df['Route_Dir_QSI_No'].str.match(pattern_A)]['ID'])]
        buses_observed_df_B = buses_observed_df[buses_observed_df['Stop Point'].isin(combined_df[combined_df['Route_Dir_QSI_No'].str.match(pattern_B)]['ID'])]

        sum_WAWT_A = buses_observed_df_A['WAWT'].sum()
        sum_buses_observed_A = buses_observed_df_A['Num of Buses Observed'].sum()
        route_AWT_A = round(sum_WAWT_A / sum_buses_observed_A, 2) if sum_buses_observed_A > 0 else 0

        sum_WAWT_B = buses_observed_df_B['WAWT'].sum()
        sum_buses_observed_B = buses_observed_df_B['Num of Buses Observed'].sum()
        route_AWT_B = round(sum_WAWT_B / sum_buses_observed_B, 2) if sum_buses_observed_B > 0 else 0

        current_hour = datetime.now(pytz.timezone('Europe/London')).hour

        # Check if the current hour's data is already present
        if current_hour in route_awt_df['Hour'].values:
            route_awt_df.loc[route_awt_df['Hour'] == current_hour, ['Route AWT A', 'Route AWT B']] = [route_AWT_A, route_AWT_B]
        else:
            new_row = pd.DataFrame({
                'Hour': [current_hour],
                'Route AWT A': [route_AWT_A],
                'Route AWT B': [route_AWT_B]
            })
            route_awt_df = pd.concat([route_awt_df, new_row], ignore_index=True)

        #print("\nRoute SWT DataFrame:")
        #print(route_swt_df)

        print("\nRoute AWT DataFrame:")
        print(route_awt_df)
        route_ewt_df = pd.DataFrame(columns=['Hour', 'Route EWT A', 'Route EWT B'])
        current_hour = datetime.now(pytz.timezone('Europe/London')).hour
        route_ewt_df = pd.DataFrame([[current_hour, None, None]], columns=['Hour', 'Route EWT A','Route EWT B'])

        # Merge route_awt_df and route_swt_df on 'Hour'
        merged_df = pd.merge(route_awt_df, route_swt_df, on='Hour')
        print(merged_df)

        # Calculate the 'Route EWT A' and 'Route EWT B' columns
        merged_df['Route EWT A'] = merged_df['Route AWT A'] - merged_df['Route SWT A']
        merged_df['Route EWT B'] = merged_df['Route AWT B'] - merged_df['Route SWT B']

        # Update or insert the data for the current hour
        if current_hour in route_ewt_df['Hour'].values:
            route_ewt_df.loc[route_ewt_df['Hour'] == current_hour, ['Route EWT A', 'Route EWT B']] = [
                merged_df.loc[merged_df['Hour'] == current_hour, 'Route EWT A'].values[0],
                merged_df.loc[merged_df['Hour'] == current_hour, 'Route EWT B'].values[0]
            ]
        else:
            new_row = pd.DataFrame({
                'Hour': [current_hour],
                'Route EWT A': [merged_df.loc[merged_df['Hour'] == current_hour, 'Route EWT A'].values[0]],
                'Route EWT B': [merged_df.loc[merged_df['Hour'] == current_hour, 'Route EWT B'].values[0]]
            })
            route_ewt_df = pd.concat([route_ewt_df, new_row], ignore_index=True)

        # Keep only the desired columns
        route_ewt_df = route_ewt_df[['Hour', 'Route EWT A', 'Route EWT B']]

        print("\nRoute EWT DataFrame:")
        print(route_ewt_df)

        print("Predictions\n")

        # Function to simulate historical data (can be replaced with real historical data loading)
        def simulate_historical_data(current_value, num_points=10):
            np.random.seed(0)  # For reproducibility
            historical_data = [current_value + np.random.normal(0, 0.1) for _ in range(num_points)]
            return pd.Series(historical_data)

        # Function to prepare data for Random Forest
        def prepare_data_for_rf(historical_series):
            X = np.arange(len(historical_series)).reshape(-1, 1)  # Time index as feature
            y = historical_series.values
            return X, y
        # Prepare for Random Forest Model
        current_hour = datetime.now(pytz.timezone('Europe/London')).hour
        all_forecast_results = []
        for _, row in buses_observed_df.iterrows():
            stop_point = row['Stop Point']
            current_awt = row['AWT']

            # Simulate historical data for the stop point
            historical_series = simulate_historical_data(current_awt)

            # Prepare data for Random Forest
            X, y = prepare_data_for_rf(historical_series)

            # Train-Test Split for Random Forest
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

            # Fit Random Forest model
            model = RandomForestRegressor(n_estimators=100, random_state=0)
            model.fit(X_train, y_train)

            # Forecast for the next 2 hours
            future_hours = np.array([[current_hour + 1]])
            # Adjust for 24-hour format
            future_hours = np.mod(future_hours, 24)
            forecast_mean = model.predict(future_hours)

            # Store forecast results
            forecast_results = pd.DataFrame({
                'Stop Point': [stop_point] * len(future_hours.flatten()) ,
                'Hour': future_hours.flatten(),
                'Forecasted AWT': forecast_mean.round(2)
            })
            all_forecast_results.append(forecast_results)
            #print(f"\nForecast results for stop point {stop_point}:")
            #print(forecast_results.to_string(index=False))
        # Concatenate all forecast results into one DataFrame
        final_forecast_df = pd.concat(all_forecast_results, ignore_index=True)
        combined_final_forecast_df = pd.merge(final_forecast_df, buses_observed_df[['Stop Point', 'Num of Buses Observed']], on='Stop Point')
        combined_final_forecast_df['Forecast WAWT'] = combined_final_forecast_df['Forecasted AWT']* combined_final_forecast_df['Num of Buses Observed']

        buses_forecasted_df_A = combined_final_forecast_df[combined_final_forecast_df['Stop Point'].isin(combined_df[combined_df['Route_Dir_QSI_No'].str.match(pattern_A)]['ID'])]
        buses_forecasted_df_B = combined_final_forecast_df[combined_final_forecast_df['Stop Point'].isin(combined_df[combined_df['Route_Dir_QSI_No'].str.match(pattern_B)]['ID'])]

        sum_forecasted_WAWT_A = buses_forecasted_df_A['Forecast WAWT'].sum()
        sum_buses_forecasted_A = buses_forecasted_df_A['Num of Buses Observed'].sum()
        route_AWT_A_forecasted = round(sum_forecasted_WAWT_A / sum_buses_forecasted_A, 2) if sum_buses_forecasted_A > 0 else 0

        sum_forecasted_WAWT_B = buses_forecasted_df_B['Forecast WAWT'].sum()
        sum_buses_forecasted_B = buses_forecasted_df_B['Num of Buses Observed'].sum()
        route_AWT_B_forecasted = round(sum_forecasted_WAWT_B / sum_buses_forecasted_B, 2) if sum_buses_forecasted_B > 0 else 0

        current_hour = datetime.now(pytz.timezone('Europe/London')).hour
        upcoming_hour = current_hour + 1

        global route_awt_forecasted_df
        if 'route_awt_forecasted_df' not in globals():
          route_awt_forecasted_df = pd.DataFrame(columns=['Hour', 'LineID', 'Route AWT A forecasted', 'Route AWT B forecasted'])
        # Check if the current hour's data is already present
        if upcoming_hour in route_awt_forecasted_df['Hour'].values:
            route_awt_forecasted_df.loc[route_awt_forecasted_df['Hour'] == upcoming_hour, ['LineID','Route AWT A forecasted', 'Route AWT B forecasted']] = [lineID,route_AWT_A_forecasted, route_AWT_B_forecasted]
        else:
            new_row = pd.DataFrame({
                'Hour': [upcoming_hour],
                'Line': [lineID],
                'Route AWT A forecasted': [route_AWT_A_forecasted],
                'Route AWT B forecasted': [route_AWT_B_forecasted]
            })
            route_awt_forecasted_df = pd.concat([route_awt_forecasted_df, new_row], ignore_index=True)

        #print("\nAll forecast results combined:")
        #print(final_forecast_df.to_string(index=False))
        print(combined_final_forecast_df)

        print("\nRoute AWT Prediction DataFrame:")
        print(route_awt_forecasted_df)
if __name__ == "__main__":
    main()



Today is Tuesday. The selected Schedule name is Monday to Friday.

Arrival Predictions for stop point 490011107G (Poplar / All Saints Church):
Line Vehicle ID Stop Point Direction Expected Arrival (BST) Expected Arrival (HM)  Gap  2_Gap  Gap_Sq
  D7    LX61DBV 490011107G  outbound    2024-08-06 14:22:36   1900-01-01 14:22:00  0.0    0.0     0.0
  D7    LX61DBU 490011107G  outbound    2024-08-06 14:34:34   1900-01-01 14:34:00 12.0   24.0   144.0
  D7    LX11BJU 490011107G  outbound    2024-08-06 14:46:32   1900-01-01 14:46:00 12.0   24.0   144.0

Cumulative DataFrame:
Line Vehicle ID Stop Point Direction Expected Arrival (BST) Expected Arrival (HM)   Gap  2_Gap  Gap_Sq
  D7    LX61DBV 490011107G  outbound    2024-08-06 14:22:36   1900-01-01 14:22:00  0.00   0.00    0.00
  D7    LX61DBU 490011107G  outbound    2024-08-06 14:34:34   1900-01-01 14:34:00 11.97  23.94  143.28
  D7    LX11BJU 490011107G  outbound    2024-08-06 14:46:32   1900-01-01 14:46:00 11.97  23.94  143.28

Number of bu

<ipython-input-2-4733df131bdc>:341: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  route_awt_df = pd.concat([route_awt_df, new_row], ignore_index=True)
<ipython-input-2-4733df131bdc>:460: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  route_awt_forecasted_df = pd.concat([route_awt_forecasted_df, new_row], ignore_index=True)


    Stop Point  Hour  Forecasted AWT  Num of Buses Observed  Forecast WAWT
0   490011107G    15            6.01                      3          18.03
1   490013513S    15            6.13                      2          12.26
2   490002048Z    15            6.10                      2          12.20
3   490006092N    15            7.20                      2          14.40
4   490004584N    15            6.62                      2          13.24
5   490015151H    15           10.53                      2          21.06
6   490004584S    15           10.64                      2          21.28
7   490000038F    15            0.03                      1           0.03
8   490006092S    15            7.95                      2          15.90
9   490002048X    15            7.84                      2          15.68
10  490013513N    15            6.60                      3          19.80

Route AWT Prediction DataFrame:
  Hour LineID  Route AWT A forecasted  Route AWT B forecasted Line


KeyboardInterrupt: 